In [1]:
import re
import pandas as pd 

import requests
from bs4 import BeautifulSoup

In [2]:
page = [ i for i in range(1, 11)]

list_kdi_news = [ ]
for p in page :
    url_kdi = 'https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg='+str(p)+'&pp=20'
    list_kdi_news.append(url_kdi)

list_kdi_news

['https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg=1&pp=20',
 'https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg=2&pp=20',
 'https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg=3&pp=20',
 'https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg=4&pp=20',
 'https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg=5&pp=20',
 'https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg=6&pp=20',
 'https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg=7&pp=20',
 'https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg=8&pp=20',
 'https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg=9&pp=20',
 'https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg=10&pp=20']

In [3]:
r = requests.get(list_kdi_news[0], verify = False) # SSLError: HTTPSConnectionPool -> verify = False 하니 실행 OK
soup = BeautifulSoup(r.text, 'html.parser')

C:\Users\y0918\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eiec.kdi.re.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


* BeautifulSoup 의 Select 사용법 -> 태그를 찾아서 -> 해당 태그들을 list에 넣고 뽑아서 나눠서 보자!

* source code : 위키북스 - 직장인을 위한 데이터 분석 실무 with 파이썬 (ch6. 어떤 무선 청소기가 좋을까?) 참고

In [4]:
news_items = soup.select('div.lb_list_com > ul > li')
len(news_items)

20

In [18]:
news_items[0]

<li id="row_225800" onclick="readCookie('225800')">
<a href="./materialView.do?num=225800&amp;topic=&amp;pp=20&amp;datecount=&amp;recommend=&amp;pg=1">
<strong>인천~서울 지하고속도로 등 6개 사업 예타 대상사업 선정 지능형 농기계 실증단지 등 3개 사업 예타 통과</strong>
<span><em>기획재정부</em> <em>2022.05.02</em> <em>10p</em></span>
</a>
<div class="lb_fil-wrap">
<button class="file_open" onclick="eiecJS.fileOpen(this)" type="button">파일</button>
<div class="lb_fil-pop">
<button onclick="eiecJS.fileClose(this)" type="button">닫기</button>
<div class="fil_scroll">
<ul>
<li><a href="/policy/callDownload.do?num=225800&amp;filenum=1&amp;dtime=20220502231752">R2205002.hwpx</a></li>
</ul>
</div>
</div>
</div>
</li>

In [19]:
title = news_items[0].select('strong')[0].text # strong 태그 있는 걸 찾아서 text로 반환해라!

title

'인천~서울 지하고속도로 등 6개 사업 예타 대상사업 선정 지능형 농기계 실증단지 등 3개 사업 예타 통과'

In [20]:
news_items[0].select('span > em')

[<em>기획재정부</em>, <em>2022.05.02</em>, <em>10p</em>]

In [8]:
dep = news_items[0].select('span > em')[0].text

dep

'기획재정부'

In [9]:
date = news_items[0].select('span > em')[1].text

date

'2022.05.02'

In [10]:
page = news_items[0].select('span > em')[2].text

page

'10p'

In [21]:
news_items[0].select('li > a')

[<a href="./materialView.do?num=225800&amp;topic=&amp;pp=20&amp;datecount=&amp;recommend=&amp;pg=1">
 <strong>인천~서울 지하고속도로 등 6개 사업 예타 대상사업 선정 지능형 농기계 실증단지 등 3개 사업 예타 통과</strong>
 <span><em>기획재정부</em> <em>2022.05.02</em> <em>10p</em></span>
 </a>,
 <a href="/policy/callDownload.do?num=225800&amp;filenum=1&amp;dtime=20220502231752">R2205002.hwpx</a>]

In [22]:
file_name = news_items[0].select('li > a')[1].text
file_name

'R2205002.hwpx'

In [23]:
text = str(  news_items[0] )

p = re.search( 'li id=.*', text )
news_num = p.group().split('=')[1].split('"')[1].split('_')[1]

news_num

'225800'

In [24]:
url_tmpl = 'https://eiec.kdi.re.kr/policy/callDownload.do?num={news_num}&filenum=1'

url_tmpl.format(news_num = news_num)

'https://eiec.kdi.re.kr/policy/callDownload.do?num=225800&filenum=1'

# 한 페이지를 작업해볼까!

In [25]:
page_num = 1

titles = [ ]
deps = [ ]
dates = [ ]
pages = [ ]
file_names = [ ]
urls = [ ]

for n in range(20) :

    titles.append( news_items[n].select('strong')[0].text )
    deps.append( news_items[n].select('span > em')[0].text )
    dates.append( news_items[n].select('span > em')[1].text )
    pages.append( news_items[n].select('span > em')[2].text )
    file_names.append( news_items[n].select('li > a')[1].text )

    text = str( news_items[n] )
    p = re.search( 'li id=.*', text )
    news_num = p.group().split('=')[1].split('"')[1].split('_')[1]
    
    urls.append( url_tmpl.format( news_num = news_num, page_num = page_num))

df = pd.DataFrame( data  = { 'title' : titles , 
                            'dep' : deps, 
                            'date' : dates, 
                            'page' : pages, 
                            'file_name' : file_names, 
                            'url' : urls } ) 

df.head()

,title,dep,date,page,file_name,url
0,인천~서울 지하고속도로 등 6개 사업 예타 대상사업 선정 지능형 농기계 실증단지 등...,기획재정부,2022.05.02,10p,R2205002.hwpx,https://eiec.kdi.re.kr/policy/callDownload.do?...
1,"’21년 귀속 양도소득세 확정신고, 5.31.까지 신고하세요",국세청,2022.05.02,39p,R2205006.hwpx,https://eiec.kdi.re.kr/policy/callDownload.do?...
2,2022년 3월 온라인쇼핑동향,통계청,2022.05.02,63p,D2205001.hwpx,https://eiec.kdi.re.kr/policy/callDownload.do?...
3,2022년4월 수출입 동향,산업통상자원부,2022.05.02,36p,R2205012.hwp,https://eiec.kdi.re.kr/policy/callDownload.do?...
4,안전한 지역축제 개최를 통해 일상회복 앞당긴다,행정안전부,2022.05.02,2p,R2205053.hwpx,https://eiec.kdi.re.kr/policy/callDownload.do?...


# 한방에 해보려는 데 자꾸 마지막 페이지만 나와! 왜지???

-> 조건문에서 계속 list들을 reset해서 빈 리스트 만들고 다시 채우니 그렇지!

In [26]:
page = [ i for i in range(1, 11)]

for p in page :

    url_kdi = 'https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg='+str(p)+'&pp=20'

    r = requests.get(url_kdi, verify = False)
    soup = BeautifulSoup(r.text, 'html.parser')

    news_items = soup.select('div.lb_list_com > ul > li')

    page_num = p

    url_tmpl = 'https://eiec.kdi.re.kr/policy/callDownload.do?num={news_num}&filenum=1'

    titles = [ ]
    deps = [ ]
    dates = [ ]
    pages = [ ]
    file_names = [ ]
    urls = [ ]

    for n in range(20) :

        titles.append( news_items[n].select('strong')[0].text )
        deps.append( news_items[n].select('span > em')[0].text )
        dates.append( news_items[n].select('span > em')[1].text )
        pages.append( news_items[n].select('span > em')[2].text )
        file_names.append( news_items[n].select('li > a')[1].text )

        text = str( news_items[n] )
        t = re.search( 'li id=.*', text )
        news_num = t.group().split('=')[1].split('"')[1].split('_')[1]

        urls.append( url_tmpl.format( news_num = news_num, page_num = page_num))

        df_n = pd.DataFrame( data  = { 'title' : titles , 
                                      'dep' : deps, 
                                      'date' : dates, 
                                      'page' : pages, 
                                      'file_name' : file_names, 
                                      'url' : urls } ) 

C:\Users\y0918\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eiec.kdi.re.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\y0918\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eiec.kdi.re.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\y0918\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eiec.kdi.re.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\y0918\anaconda3\lib\site-packages\urllib3\connectionpoo

In [27]:
 df_n.head()

,title,dep,date,page,file_name,url
0,공공부문의 장애인 표준사업장 생산품 구매 지속 증가,고용노동부,2022.04.27,6p,R2204905.hwpx,https://eiec.kdi.re.kr/policy/callDownload.do?...
1,2022년 3월 국내인구이동통계,통계청,2022.04.27,14p,D2204009.pdf,https://eiec.kdi.re.kr/policy/callDownload.do?...
2,2022년도 공시대상기업집단 76개 지정,공정거래위원회,2022.04.27,17p,R2204919.hwpx,https://eiec.kdi.re.kr/policy/callDownload.do?...
3,「교육관련기관의 정보공개에 관한 특례법 시행령」 일부개정령안 국무회의 통과,교육부,2022.04.27,6p,R2204913.hwpx,https://eiec.kdi.re.kr/policy/callDownload.do?...
4,벤츠·랜드로버·기아·벤틀리·피라인 시정조치(리콜),국토교통부,2022.04.27,9p,R2204898.hwp,https://eiec.kdi.re.kr/policy/callDownload.do?...


In [28]:
df_n.shape

(20, 6)

# 이제 한방에 해볼까?

In [29]:
import warnings
warnings.filterwarnings('ignore')

In [30]:
import re
import pandas as pd 
from tqdm import tqdm

import requests
from bs4 import BeautifulSoup

In [31]:
page = [ i for i in range(1, 31) ] # 30페이지까지 도전해보자!

for p in tqdm(page) :
    url_kdi = 'https://eiec.kdi.re.kr/policy/materialList.do?topic=&pg='+str(p)+'&pp=20'

    r = requests.get(url_kdi, verify = False)
    soup = BeautifulSoup(r.text, 'html.parser')

    news_items = soup.select('div.lb_list_com > ul > li')

    page_num = p
    url_tmpl = 'http://eiec.kdi.re.kr/policy/callDownload.do?num={news_num}&filenum=1'

    if p == 1 : 
        titles = [ ]
        deps = [ ]
        dates = [ ]
        pages = [ ]
        file_names = [ ]
        urls = [ ]

    for n in range(20) :
        titles.append( news_items[n].select('strong')[0].text )
        deps.append( news_items[n].select('span > em')[0].text )
        dates.append( news_items[n].select('span > em')[1].text )
        pages.append( news_items[n].select('span > em')[2].text )
        file_names.append( news_items[n].select('li > a')[0].text )

        text = str(  news_items[n] )
        t = re.search( 'li id=.*', text )
        news_num = t.group().split('=')[1].split('"')[1].split('_')[1]

        urls.append( url_tmpl.format( news_num = news_num, page_num = page_num))

    else :
        for n in range(20) :
            titles.append( news_items[n].select('strong')[0].text )
            deps.append( news_items[n].select('span > em')[0].text )
            dates.append( news_items[n].select('span > em')[1].text )
            pages.append( news_items[n].select('span > em')[2].text )
            file_names.append( news_items[n].select('li > a')[0].text )

            text = str(  news_items[n] )
            t = re.search( 'li id=.*', text )
            news_num = t.group().split('=')[1].split('"')[1].split('_')[1]

            urls.append( url_tmpl.format( news_num = news_num, page_num = page_num))

df = pd.DataFrame( data  = { 'title' : titles , 
                            'dep' : deps, 
                            'date' : dates, 
                            'page' : pages, 
                            'file_name' : file_names, 
                            'url' : urls } ) 


df.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:30<00:00,  1.00s/it]


,title,dep,date,page,file_name,url
0,인천~서울 지하고속도로 등 6개 사업 예타 대상사업 선정 지능형 농기계 실증단지 등...,기획재정부,2022.05.02,10p,\n인천~서울 지하고속도로 등 6개 사업 예타 대상사업 선정 지능형 농기계 실증단지...,http://eiec.kdi.re.kr/policy/callDownload.do?n...
1,"’21년 귀속 양도소득세 확정신고, 5.31.까지 신고하세요",국세청,2022.05.02,39p,"\n’21년 귀속 양도소득세 확정신고, 5.31.까지 신고하세요\n국세청 2022....",http://eiec.kdi.re.kr/policy/callDownload.do?n...
2,2022년 3월 온라인쇼핑동향,통계청,2022.05.02,63p,\n2022년 3월 온라인쇼핑동향\n통계청 2022.05.02 63p\n,http://eiec.kdi.re.kr/policy/callDownload.do?n...
3,2022년4월 수출입 동향,산업통상자원부,2022.05.02,36p,\n2022년4월 수출입 동향\n산업통상자원부 2022.05.02 36p\n,http://eiec.kdi.re.kr/policy/callDownload.do?n...
4,안전한 지역축제 개최를 통해 일상회복 앞당긴다,행정안전부,2022.05.02,2p,\n안전한 지역축제 개최를 통해 일상회복 앞당긴다\n행정안전부 2022.05.02 ...,http://eiec.kdi.re.kr/policy/callDownload.do?n...


In [32]:
df.shape

(1200, 6)

In [31]:
df.to_excel('df_kdi_news.xlsx')

# 다운로드 작업은 로컬 PC에서!